In [2]:
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from hackutilitiy import *

In [ ]:
# Load the data from Parquet files into Polars lazy frame
txt_features_lf = txt_features_parquet_to_lf()
# Tiny peek that won't crash (evaluates only N rows):
print(txt_features_lf.tail(50).collect())

shape: (50, 8)
┌──────────┬─────────┬───────────┬──────────┬───────────┬──────┬───────────────┬─────────────────┐
│ date     ┆ cik     ┆ file_type ┆ gvkey    ┆ cusip     ┆ year ┆ rf_word_count ┆ mgmt_word_count │
│ ---      ┆ ---     ┆ ---       ┆ ---      ┆ ---       ┆ ---  ┆ ---           ┆ ---             │
│ str      ┆ i64     ┆ str       ┆ f64      ┆ str       ┆ i64  ┆ u32           ┆ u32             │
╞══════════╪═════════╪═══════════╪══════════╪═══════════╪══════╪═══════════════╪═════════════════╡
│ 20250530 ┆ 102037  ┆ 10K       ┆ 11017.0  ┆ 913456109 ┆ 2025 ┆ 6911          ┆ 39              │
│ 20250530 ┆ 109198  ┆ 10Q       ┆ 11672.0  ┆ 872540109 ┆ 2025 ┆ 48            ┆ 4653            │
│ 20250530 ┆ 357294  ┆ 10Q       ┆ 5750.0   ┆ 442487401 ┆ 2025 ┆ 0             ┆ 10438           │
│ 20250530 ┆ 711404  ┆ 10Q       ┆ 3504.0   ┆ 216648501 ┆ 2025 ┆ 140           ┆ 3816            │
│ 20250530 ┆ 831489  ┆ 10Q       ┆ null     ┆ null      ┆ 2025 ┆ 8             ┆ 1700         

In [ ]:
# Count the number of rows in the lazy frame
# This will trigger a computation but is efficient
# as it only computes the length without loading all data into memory.
n_rows = txt_features_lf.select(pl.len()).collect().item()  # returns Python int
print(n_rows)

358439


In [ ]:
# Compute the text features by reading Duckdb shards of the parquet file and converting to CSV
# the only fix that did not crash my system
txt_features_parquet_to_csv()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Done. Wrote: data/txt_features.csv


## Inspect the rf and mgmt reports

In [53]:
rf_sample1 = txt_sample.rf.reset_index(drop=True).dropna().iloc[16542]

In [54]:
rf_sample2 = txt_sample.rf.reset_index(drop=True).dropna().iloc[16535]

In [55]:
print('\n'.join(textwrap.fill(para, width=80) for para in rf_sample1.split('\n') if para.strip()))

Item 1A. Risk Factors 37 ITEM 1A. RISK FACTORS.
OUR OPERATIONS ARE SUBJECT TO INTEREST RATE RISK AND VARIATIONS IN INTEREST
RATES MAY NEGATIVELY AFFECT FINANCIAL PERFORMANCE.
 Our earnings and cash flows are largely dependent upon our net interest
income. Net interest income is the difference between interest income earned on
interest-earning assets such as loans and securities and interest expense paid
on interest-bearing liabilities such as deposits and borrowed money. Changes in
the general level of interest rates may have an adverse effect on our business,
financial condition and result of operations. Interest rates are highly
sensitive to many factors that are beyond our control, including general
economic conditions and policies of various governmental and regulatory agencies
and, in particular, the FRB. Changes in monetary policy, including changes in
interest rates, influence the amount of interest income that we receive on loans
and securities and the amount of interest that w

In [56]:
print('\n'.join(textwrap.fill(para, width=80) for para in rf_sample2.split('\n') if para.strip()))

Item 1A. Risk
Factors
 Because of the following
factors, as well as other factors affecting the Company s operating results and
financial condition, past financial performance should not be considered to be
a reliable indicator of future performance, and investors should not use
historical trends to anticipate results or trends in future periods.
 The matters relating to the investigation by the
Special Committee of the Board of Directors and the restatement of the Company s
consolidated financial statements may result in additional litigation and
governmental
enforcement actions.
 On June 29, 2006, the Company announced that an
internal review had discovered irregularities related to the issuance of
certain stock option grants made between 1997 and 2001, including a grant to
its Chief Executive Officer CEO ), Steve Jobs. The Company also announced a
Special Committee of outside directors Special Committee had been formed
and had hired independent counsel to conduct a full investigatio

In [57]:
mgmt_sample1 = txt_sample.mgmt.reset_index(drop=True).dropna().iloc[11]
mgmt_sample2 = txt_sample.mgmt.reset_index(drop=True).dropna().iloc[73]

In [58]:
print('\n'.join(textwrap.fill(para, width=80) for para in mgmt_sample1.split('\n') if para.strip()))

Item 2.
Management s Discussion and Analysis of Financial Condition and Results of
Operations
 12 Item 2. Management s Discussion and Analysis of Financial Condition and
Results of Operations Forward-Looking Statements This discussion contains
forward-looking statements within the meaning of the Private Securities
Litigation Reform Act of 1995. These
statements reflect our current views with respect to future events and financial
performance. The words believe, expect, anticipate, intend, estimate, forecast,
 project, should and similar expressions are intended to identify forward-
looking statements within the meaning of the Private Securities Litigation
Reform Act of 1995. All forecasts and projections in this
document are forward-looking statements, and are based on management s current
expectations or beliefs of the Company s results, based on current information
available pertaining to the Company, including the risk factors noted
below. From time to time, we also may provide oral

In [59]:
print('\n'.join(textwrap.fill(para, width=80) for para in mgmt_sample2.split('\n') if para.strip()))

Item 2 Management s Discussion and Analysis of Financial Condition and Results
of Operations Item 2 Management s Discussion and Analysis of Financial Condition
and Results of Operations
Cautionary Disclosures To Qualify Forward Looking Statements
This report on Form 10-Q includes forward-looking statements within the meaning
of Section
27A of the Securities Act of 1933, as amended, and Section 21E of the Securities
Exchange Act of
1934, as amended. All statements other than statements of historical facts
included in this Form
10-Q, including, without limitation, statements contained in this Management s
Discussion and
Analysis of Financial Condition and Results of Operations and Notes to
Consolidated Financial
Statements located elsewhere in this report regarding our financial position,
business strategy,
plans and objectives of management for future operations, future sales and
industry conditions, are
forward-looking statements. Although we believe that the expectations reflected
in 

# Construct Text data dataframe

In [60]:
txt_db = txt_sample.loc[:,["year", "date", "file_type", "rf", "mgmt", "gvkey", "cusip", "cik" ]].copy()
txt_db.tail()

,year,date,file_type,rf,mgmt,gvkey,cusip,cik
213523,2006,2006-12-29,10K,ITEM 1A. \n\nRISK\n FACTORS \n\n13 Item 1A. \n...,ITEM 7. \n\nMANAGEMENT S\n DISCUSSION AND ANAL...,113361,149016107,1063085
213506,2006,2006-12-29,10K,Item\n 1A. \n\nRisk\n Factors \n\n1.\n We May ...,Item\n 7. \n\nManagement s\n Discussion and An...,147249,47077R109,1133062
213509,2006,2006-12-29,10K,ITEM 1A \n\nRISK FACTORS \n \n 11 ITEM 1A. Ris...,ITEM 7. \n\nMANAGEMENT S DISCUSSION AND ANALYS...,230796,00506P103,1183941
213524,2006,2006-12-29,10K,ITEM 1A. RISK FACTORS Risk Factors This report...,ITEM 7. MANAGEMENT S DISCUSSION AND ANALYSIS O...,162956,00430L103,1310094
213529,2006,2006-12-29,10K,Item 1A. Risk Factors 33 ITEM 1A. RISK FACTORS...,Item 7. Management's Discussion and Analysis o...,162576,114039100,1310313


In [42]:
data_dct['global_name_merge'].columns

Index(['fic', 'gvkey', 'date', 'iid', 'conm'], dtype='object')

In [61]:
txt_db.merge(data_dct['global_name_merge'].drop_duplicates(subset=["gvkey", "date"]),
              how='left', 
              on =["gvkey",  "date"], 
              validate="one_to_one").dropna()

,year,date,file_type,rf,mgmt,gvkey,cusip,cik,fic,iid,conm
8498,2006,2006-06-30,10Q,,Item 2. Management's Discussion and Analysis o...,13498,143658300,815097,PAN,01W,CARNIVAL CORPORATION & PLC


In [62]:
data_dct['na_name_merge'].columns

Index(['gvkey', 'date', 'iid', 'tic', 'cusip', 'conm', 'cik'], dtype='object')

In [63]:
txt_db.merge(data_dct['na_name_merge'].drop_duplicates(subset=["gvkey", "cik", "cusip", "date"]),
              how='left', on =["gvkey", "cik", "cusip", "date"]).dropna().drop(['cik', 'cusip', 'gvkey', 'iid'], axis=1).reset_index(drop=True)   

,year,date,file_type,rf,mgmt,tic,conm
0,2006,2006-01-31,10Q,,Item 2. Management's Discussion and Analysis o...,SCX,STARRETT (L.S.) CO -CL A
1,2006,2006-01-31,10Q,,Item 2. Management s Discussion and Analysis o...,WWD,WOODWARD INC
2,2006,2006-01-31,10Q,,Item 2. \n\nManagement s Discussion and Analys...,CVCO,CAVCO INDUSTRIES INC
3,2006,2006-01-31,10K,,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,ANIX,ANIXA BIOSCIENCES INC
4,2006,2006-01-31,10Q,Item 1A Risk Factors 16 Item 1A Risk Factors\n...,Item 2 Management's Discussion and Analysis of...,UCIA,UCI MEDICAL AFFILIATES INC
...,...,...,...,...,...,...,...
591,2006,2006-11-30,10K,Item 1A Risk Factors 2 ITEM 1A. RISK FACTORS\n...,Item 7 Management's Discussion and Analysis of...,OFI,OVERHILL FARMS INC
592,2006,2006-11-30,10Q,Item 1A. Risk Factors Item 1A. Risk Factors \n...,Item 2. Management s Discussion and Analysis o...,AVNT,AVIENT CORP
593,2006,2006-11-30,10Q,Item 1A. Risk Factors............................,Item 2. Management's Discussion and Analysis o...,CONNQ,CONN'S INC
594,2006,2006-11-30,10Q,ITEM 1A. \n \n RISK FACTORS We have disclosed ...,ITEM 2.\n MANAGEMENT S DISCUSSION AND ANALYSIS...,EXLS,EXLSERVICE HOLDINGS INC


In [159]:
data_dct['us_cik_gvkey_linktable'].columns

Index(['gvkey', 'date', 'iid', 'tic', 'cusip', 'conm', 'tpci', 'cik'], dtype='object')

In [64]:
txt_db.merge(data_dct['us_cik_gvkey_linktable'].drop_duplicates(subset=["gvkey", "cik", "cusip", "date"]),
              how='left', 
              on =["gvkey", "cik", "cusip", "date"]).dropna().drop(['cik', 'cusip', 'gvkey', 'iid'], axis=1).reset_index(drop=True).iloc[300:400]  

,year,date,file_type,rf,mgmt,tic,conm,tpci
300,2006,2006-03-31,10K,Item 1A. \n\nRisk Factors \n\n20 ITEM 1A. RISK...,Item 7. \n\nManagement s Discussion and Analys...,SNDA,SONIDA SENIOR LIVING INC,0
301,2006,2006-03-31,10K,ITEM 1A. \n\nRisk Factors \n \n 6 Item 1A. Ris...,ITEM 7. \n\nManagement s Discussion and Analys...,VERT,VERTICALNET INC,0
302,2006,2006-03-31,10K,Item 1A. \n\nRisk\n Factors \n\nWe\n have a hi...,Item 7. \n\nManagement s\n Discussion and Anal...,ENHT,ENHERENT CORP,0
303,2006,2006-03-31,10K,Item\n 1A. Risk Factors \n\nAn\n investment in...,Item\n 7. Management's Discussion and Analysis...,GFED,GUARANTY FED BANCSHARES INC,0
304,2006,2006-03-31,10K,Item 1A. \n\nRisk Factors \n \n 15 ITEM 1A. RI...,Item 7. \n\nManagement s Discussion and Analys...,OMNI,OMNI ENERGY SERVICES CORP,0
...,...,...,...,...,...,...,...,...
395,2006,2006-03-31,10K,Item 1A \n \n Risk Factors \n\n40 Item 1A. RIS...,Item 7 \n \n Management s Discussion and Analy...,BKD,BROOKDALE SENIOR LIVING INC,0
396,2006,2006-03-31,10K,Item 1A \n\nRisk Factors \n\n14 Item 1A. \n \n...,Item 7 \n\nManagement s Discussion and Analysi...,CBOU,CARIBOU COFFEE CO,0
397,2006,2006-03-31,10K,Item 1A \n\nRisk Factors \n\n7 Item 1A. \n\nRi...,Item 7 \n\nManagement s\n Discussion and Analy...,CSA.3,COGDELL SPENCER INC,0
398,2006,2006-03-31,10K,Item 1A Risk Factors \n\n16 Item 1A. Risk Fact...,Item 7 Management s Discussion and Analysis of...,TCMIQ,TRIPLE CROWN MEDIA INC,0


🔹 cik

Meaning: Central Index Key.

Issued by the SEC’s EDGAR system to uniquely identify companies and individuals who file disclosure documents with the SEC.

Example: Apple’s CIK = 0000320193.

Used to link Compustat firms (gvkey) with SEC filings (10-K, 10-Q).

🔹 tpci

Meaning: Trading Public Company Indicator (Compustat variable).

A flag indicating the trading status of the security or company.

Values (from WRDS / Compustat docs):

"0" = Not a public company / not trading

"1" = Publicly traded company

Sometimes extended values exist depending on dataset vintage.

🔹 In your us_cik_gvkey_linktable context

gvkey → Compustat’s unique firm identifier.

cik → SEC’s identifier (so you can join to EDGAR filings).

tpci → Indicator whether the gvkey is linked to a publicly traded company (helps filter out private or non-trading entities when consolidating).

✅ So practically:

Use cik when you want to pull SEC filings for the firm.

Use tpci to filter down to actively traded companies (exclude stale or private).




🔹 cusip

Meaning: Committee on Uniform Securities Identification Procedures number.

It’s a 9-character alphanumeric identifier assigned to U.S. and Canadian securities (stocks and bonds).

Structure:

First 6 = issuer code.

Next 2 = issue identifier.

Last 1 = check digit.

In Compustat: lets you link to market data (CRSP, Bloomberg, etc.) at the security level.

Example: Apple Inc. common stock = 037833100.

🔹 iid

Meaning: Issue Identifier (Compustat).

Distinguishes different securities (issues) for the same firm (gvkey).

Example: A company might have common stock, preferred stock, bonds — all share the same gvkey but different iid.

Together, (gvkey, iid) pinpoints a unique security within a firm.

🔹 fic

Meaning: Foreign Incorporation Code.

Indicates the country of incorporation of the firm.

It’s a 3-character ISO country code (e.g., USA, CAN, GBR).

Useful when consolidating across global datasets or filtering U.S. vs foreign companies.

✅ In summary, in this context:

cusip → security identifier, links to markets.

iid → security issue within the firm.

fic → firm’s country of incorporation.